<a href="https://colab.research.google.com/github/khushiyadav03/Machine-Learning/blob/main/Text_Data_Pre_Processing_Use_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# printing the stopwords
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Pre-Processing

In [6]:
# loading the dataset to a pandas dataframe
news_dataset = pd.read_csv('/content/fake_news_dataset.csv', encoding='latin-1', engine='python', on_bad_lines='skip')

In [7]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


0 --> Real News

1 --> Fake News

In [8]:
news_dataset.shape

(11397, 5)

In [9]:
# checking for missing values
news_dataset.isnull().sum()

,0
id,0
title,310
author,1064
text,23
label,0


In [10]:
# replacing the missing values with null string
news_dataset = news_dataset.fillna('')

In [11]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [12]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,Darrell Lucus House Dem Aide: We Didnât Even...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [14]:
# separating feature and target
X = news_dataset.drop(columns='label', axis =1)
Y = news_dataset['label']

In [15]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didnât Even See Comeyâs...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
11392  11392  Three Arrested for Starting Massive Fire on At...   
11393  11393  Oh Myâ¦ WikiLeaks Email Between Stratfor Inte...   
11394  11394                                                      
11395  11395  Comment on Censored News: Black woman kills 12...   
11396  11396  ICE Deporting 199 Iraqis after Trump Inks Repa...   

                    author                                               text  \
0            Darrell Lucus  House Dem Aide: We Didnât Even See Comeyâs...   
1          Daniel J. Flynn  Ever 

In [16]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
11392    0
11393    1
11394    1
11395    1
11396    0
Name: label, Length: 11397, dtype: int64


Stemming:

Stemming is the process of reducing a word to its Root Word

In [17]:
port_stem = PorterStemmer()

In [18]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [20]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [21]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
11392    katherin rodriguez three arrest start massiv f...
11393    ln buzz team oh wikileak email stratfor intel ...
11394                                            starmount
11395    tnb pa comment censor news black woman kill wh...
11396    bob price ice deport iraqi trump ink repatri deal
Name: content, Length: 11397, dtype: object


In [23]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [24]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ... 'starmount'
 'tnb pa comment censor news black woman kill white boy blow torch tnb pa'
 'bob price ice deport iraqi trump ink repatri deal']


In [25]:
print(Y)

[1 0 1 ... 1 1 0]


In [26]:
Y.shape

(11397,)

In [27]:
# converting the textual data to feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)


In [28]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 115415 stored elements and shape (11397, 13070)>
  Coords	Values
  (0, 220)	0.2744532428202811
  (0, 1926)	0.3683026479343919
  (0, 2291)	0.24753616371057055
  (0, 2787)	0.3559095383289156
  (0, 2937)	0.27131882903925914
  (0, 3840)	0.23783638021346015
  (0, 5399)	0.22501117885772176
  (0, 5920)	0.24582983426985566
  (0, 6628)	0.2887859359853237
  (0, 6839)	0.3559095383289156
  (0, 10306)	0.2612548913337051
  (0, 12018)	0.2855125249395007
  (1, 1179)	0.2880075654568842
  (1, 1483)	0.15153491395419627
  (1, 1728)	0.3811856274653655
  (1, 2174)	0.1885038505349307
  (1, 2763)	0.2648139521049962
  (1, 4263)	0.7222375895049578
  (1, 5264)	0.1877176969808806
  (1, 12825)	0.29370245003511364
  (2, 2277)	0.31980064355028776
  (2, 2407)	0.45732294873998436
  (2, 4177)	0.3864480997616394
  (2, 4613)	0.3454378282810494
  (2, 7366)	0.48962804150853995
  :	:
  (11393, 11531)	0.222850828972591
  (11393, 12427)	0.3139497496339345
  (11393,